In [1]:
import os 
import streamlit
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
import dotenv 
dotenv.load_dotenv()

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [2]:

from huggingface_hub import login
llm=ChatGroq(
    groq_api_key=os.getenv('GROQ_API_KEY'),
    model_name='llama-3.1-70b-versatile',
    temperature=0.2,
    
    
)
login(token=os.getenv('HUGGINGFACEHUB_API_TOKEN'),add_to_git_credential=True)

d:\anaconda\envs\genai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (manager).
Your token has been saved to C:\Users\26amr\.cache\huggingface\token
Login successful


# split the document and store it 

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, load_index_from_storage, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# import faiss
# embeddings=OllamaEmbeddings()
# loader=WebBaseLoader('https://en.wikipedia.org/wiki/Elon_Musk')
# docs=loader.load()
# text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
# documents=text_splitter.split_documents(docs)
# print(documents)

documents=SimpleDirectoryReader('../data/').load_data()
embed_model=HuggingFaceEmbedding(
    model_name='BAAI/bge-small-en-v1.5'
)
from llama_index.core import Settings

Settings.chunk_size=1024
Settings.embed_model=embed_model
Settings.llm=llm


# store or retrive the data from database

In [4]:
# index=VectorStoreIndex.from_documents(documents=documents,embed_model=Settings.embed_model)
# # save the data 
# index.storage_context.persist('../data/')
# # load index from storage
# storage_context=StorageContext.from_defaults(persist_dir='../data/')
# index=load_index_from_storage(storage_context)

# # create a retriever or query engine 
# retriever=index.as_retriever()
# query_engine=index.as_query_engine()


In [5]:
from llama_index.core import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display

# create chroma collection
db=chromadb.PersistentClient(path='../data/chroma_db/') 
# chroma_client=chromadb.EphemeralClient()
chroma_collection=db.get_or_create_collection('test')

# set up chromavector store and load in the data
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
# embed_model=LangchainEmbedding(HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))
vector_store=ChromaVectorStore(chroma_collection=chroma_collection)

storage_context=StorageContext.from_defaults(vector_store=vector_store)
index=VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

query_engine=index.as_query_engine()
response=query_engine.query('summarize the info')
display(Markdown(f"<b>{response}</b>"))




d:\anaconda\envs\genai\lib\site-packages\llama_index\llms\langchain\base.py:95: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  lc_message = self._llm.predict_messages(messages=lc_messages, **kwargs)


<b>The provided information appears to be a collection of metadata for a single PDF file, "texasLaw.pdf", which is approximately 5.53 MB in size and was created and last modified on November 7, 2024. The file is associated with multiple node IDs and page labels, ranging from page 5 to page 51. The file type is application/pdf, and the file path is consistently listed as being located in the same directory.</b>

In [6]:
print(chroma_collection.get()['documents'])

['{"embedding_dict": {"601feb9c-2221-4d41-b50e-27e22418bcab": [-0.04526807367801666, -0.015631889924407005, 0.016970276832580566, -0.014185497537255287, 0.000586961570661515, -0.027328770607709885, 0.012104304507374763, -0.00677449768409133, -0.010457870550453663, -0.012452060356736183, 0.023837709799408913, 0.04354994371533394, -0.026827268302440643, 0.011108338832855225, -0.03820706158876419, -0.02868381328880787, -0.015605204738676548, 0.02321833372116089, 0.04432843253016472, -0.0012139561586081982, 0.03034285642206669, -0.03893328458070755, 0.02465512789785862, -0.05359910801053047, 0.0684807151556015, -0.002011738019064069, -0.023131046444177628, -0.022159939631819725, -0.07667095959186554, -0.12880489230155945, 0.04110152646899223, -0.007081767078489065, -0.042449578642845154, -0.016354557126760483, 0.01513734832406044, 0.0348886176943779, 0.03151645511388779, -0.008322865702211857, -0.04052385687828064, 0.013003087602555752, 0.00015703868120908737, 0.013395214453339577, -0.0182

In [7]:
# load from db
from llama_index.core import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display

db2=chromadb.PersistentClient(path='../data/chroma_db') 
chroma_collection=db2.get_or_create_collection('test')
print(chroma_collection.get()['documents'])
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
vector_store=ChromaVectorStore(chroma_collection=chroma_collection)
index=VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)
query_engine=index.as_query_engine()
response=query_engine.query('summarize the info')
display(Markdown(f"<b>{response}</b>"))

['{"embedding_dict": {"601feb9c-2221-4d41-b50e-27e22418bcab": [-0.04526807367801666, -0.015631889924407005, 0.016970276832580566, -0.014185497537255287, 0.000586961570661515, -0.027328770607709885, 0.012104304507374763, -0.00677449768409133, -0.010457870550453663, -0.012452060356736183, 0.023837709799408913, 0.04354994371533394, -0.026827268302440643, 0.011108338832855225, -0.03820706158876419, -0.02868381328880787, -0.015605204738676548, 0.02321833372116089, 0.04432843253016472, -0.0012139561586081982, 0.03034285642206669, -0.03893328458070755, 0.02465512789785862, -0.05359910801053047, 0.0684807151556015, -0.002011738019064069, -0.023131046444177628, -0.022159939631819725, -0.07667095959186554, -0.12880489230155945, 0.04110152646899223, -0.007081767078489065, -0.042449578642845154, -0.016354557126760483, 0.01513734832406044, 0.0348886176943779, 0.03151645511388779, -0.008322865702211857, -0.04052385687828064, 0.013003087602555752, 0.00015703868120908737, 0.013395214453339577, -0.0182

<b>The provided information appears to be a collection of metadata for a PDF file named "texasLaw.pdf". The metadata includes details such as page labels, file paths, file types, file sizes, creation dates, and last modified dates. There are multiple entries with the same file name and path, but different page labels, suggesting that the file may be a multi-page document. The file size and creation/modified dates are consistent across all entries, indicating that the file has not been modified.</b>

# chat

In [8]:
query_engine=index.as_query_engine()
response=query_engine.query('what does the document talk about?')
display(Markdown(f"<b>{response}</b>"))

<b>The document appears to be discussing various aspects of law, specifically the Constitution and its sections, as well as the power of cities and towns to tax and issue bonds. It also touches on the concept of mortgages and the sale of real estate.</b>

In [9]:
## add prompts 

In [10]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.prompts import ChatPromptTemplate
from llama_index.core.memory import BaseMemory
# memory=BaseMemory()
language= 'French'
# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            """ 
            You are an experienced immigration lawyer providing detailed legal advice in {language}. 
            Carefully review the following documents to extract relevant information and address the client's question 
            Analyze any legal regulations, risks, or implications that may apply, and provide guidance on the next steps.

                Instructions:
                1. Identify key legal issues or relevant regulations from the provided documents.
                2. Explain how the information applies to the client’s situation.
                3. Highlight any significant risks, legal obligations, or considerations the client should be aware of.
                4. Offer clear advice on the next steps, considering short-term and long-term outcomes.
                5. Note any additional documentation, evidence, or forms the client should prepare

                Please provide a structured and thorough response, clearly addressing each point in a way that helps the client understand their legal standing and options in {language}.
            """
        ),
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information and not prior knowledge, "
            "answer the query.\n"
            "Query: {query_str}\n"
            "Answer: "
        ),
    ),
]

text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# Refine Prompt
chat_refine_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "You are an expert Q&A system that strictly operates in two modes "
            "when refining existing answers:\n"
            "1. **Rewrite** an original answer using the new context.\n"
            "2. **Repeat** the original answer if the new context isn't useful.\n"
            "Never reference the original answer or context directly in your answer.\n"
            "When in doubt, just repeat the original answer."
        ),
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "New Context: {context_msg}\n"
            "Query: {query_str}\n"
            "Original Answer: {existing_answer}\n"
            "New Answer: "
        ),
    ),
]
refine_template = ChatPromptTemplate(chat_refine_msgs)

In [11]:
from llama_index.core.memory import ChatMemoryBuffer

memory= ChatMemoryBuffer.from_defaults(token_limit=1000)
chat_engine=index.as_chat_engine(
    chat_mode='context',
    memory=memory,
    system_prompt=(text_qa_template,refine_template),
)


In [ ]:
response=chat_engine.chat('what was the latest law passed?')
print(response.response)

'The latest law passed mentioned in the context is the act of April 8, 1889, which clearly prescribes the limit of taxation of cities with more than ten thousand inhabitants.'

In [14]:
response=chat_engine.chat('could you provide more info')
print(response.response)

Unfortunately, the context information does not provide more details about the act of April 8, 1889. However, it does mention that the act clearly prescribes the limit of taxation of cities with more than ten thousand inhabitants.

Additionally, the context information mentions other laws and regulations, such as:

* Article X, section 3 of the Constitution, which requires railway companies to keep a public office in the state.
* Laws 1876, page 144, which requires railway companies to maintain a public office on the line of their road in the state.
* Revised Statutes, article 4115, which retains the requirement for railway companies to maintain a public office on the line of their road in the state.
* General Laws 1885, page 188, which is mentioned but not elaborated on.

If you are looking for more information on a specific law or regulation, please let me know and I will try to provide more context or clarify the information.


In [15]:
response=chat_engine.chat('what are the things i sould avoid?')
print(response.response)

Based on the context information, it appears that the following things should be avoided:

1. Disregarding one's interests and those of the state.
2. Incurring penalties from the state for securing honors from others.
3. Affecting private rights while discharging official functions.
4. Making a new qualification for holding office.
5. Preferring those who develop the country with their words over those who do so with their actions (e.g., pioneers who work the land).

Additionally, the context information mentions that a lax application of the principles announced should be avoided.

It is also worth noting that the context information does not provide a comprehensive list of things to avoid, but rather highlights specific issues related to the interpretation of laws and regulations.


In [16]:
response=chat_engine.chat('anything perticular about investing?')
print(response.response)

There is no specific information about investing in the context provided. However, there is a mention of insurance companies and taxation, which may be relevant to investing in insurance companies or related financial instruments.

According to the context, insurance companies are subject to certain taxes, including:

* Life insurance companies and life and accident insurance companies: an annual state tax of one and one-fourth of one percent on the gross amount of premiums received in the state during the present year.
* Fire, marine, health, live stock, guarantee, and accident insurance companies: an annual state tax of one-half of one percent on the gross amount of premiums received in the state during the preceding year.
* State, county, and municipal ad valorem taxes on the real and personal property of insurance companies within the state.

It's worth noting that this information is specific to the state of Texas and may not be applicable to other jurisdictions.


In [17]:
response=chat_engine.chat('tell me more about taxes and what should i avoid?')
print(response.response)

Based on the context information, here are some key points about taxes and what to avoid:

1. **Insurance companies' taxes**: Insurance companies are subject to various taxes, including state taxes on premiums received and ad valorem taxes on real and personal property.
2. **Tax rates**: The tax rates for insurance companies vary depending on the type of insurance, with life insurance companies paying a higher rate (1.25%) than other types of insurance companies (0.5%).
3. **Avoid occupation taxes**: The context information mentions that counties and municipalities are prohibited from levying occupation taxes on insurance companies.
4. **Be aware of tax exemptions**: Some entities, such as railway companies, may be exempt from certain taxes or have specific tax requirements.
5. **Understand tax implications**: When investing in or working with insurance companies, it's essential to understand the tax implications and potential liabilities.

To avoid potential issues with taxes, it's re